# Quiz Notebook 2: FEI Protocol Savings Rate

This Quiz Notebook is associated to [**Analysis Notebook 2: FEI Savings Rate**](../2_fei_savings_rate.ipynb).

The goal of this notebook is to comprehensively evaluate an analysis by way of using parameter sweeps for monetary policy setting.

# Quiz Setup 

- The first section of all quiz notebooks contains all necessary code to run the relevant simulation **within the quiz**.
- However, this is done for enabling experimentation. **In practice, each notebook should be gone through with the associated analysis notebook**.
- Analysis notebook sections relevant to quiz questions will be pointed to in the analysis notebooks directly.
- Solution cells are not meant to directly execute, as these may only contain the relevant snippets.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [ ]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import display_code

In [ ]:
import plotly.io as pio
png_renderer = pio.renderers["png"]
png_renderer.width = 1200
png_renderer.height = 500
# png_renderer.scale = 1

pio.renderers.default = "png"

In [ ]:
from operator import lt, gt

In [ ]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False

In [ ]:
# Import experiment templates
import experiments.default_experiment as default_experiment

In [ ]:
# Inspect experiment template
display_code(default_experiment)

### Goal: Understanding Fei Savings rate and Capital Allocation Model

#### Prompt:

In [ ]:
# Create a simulation for each analysis
simulation_1 = copy.deepcopy(default_experiment.experiment.simulations[0])

In [ ]:
cam_deposits = [
    "fei_liquidity_pool_user_deposit",
    "fei_money_market_user_deposit",
    "fei_savings_user_deposit",
    "fei_idle_user_deposit",
]

In [ ]:
parameter_overrides = {
    "capital_allocation_fei_deposit_variables": [
            cam_deposits,
    ],
    "capital_allocation_rebalance_duration": [30],
    "fei_savings_rate_process": [
         lambda _run, timestep: 0.005,
         lambda _run, timestep: 0.03,
         lambda _run, timestep: 0.005 if timestep < 365 / 4 else (0.03 if timestep < 365 * 3/4 else 0.02),
         lambda _run, timestep: 0,
    ],
}

In [ ]:
# Experiment configuration
simulation_1.model.runs = 1

# Override inital state
simulation_1.model.initial_state.update({})

# Override default experiment System Parameters
simulation_1.model.params.update(parameter_overrides)

In [ ]:
# Analysis-specific setup

In [ ]:
# Experiment execution
# df, exceptions = run(simulation_1)

In [ ]:
df.query('run==1').plot(x='timestamp', y=['fei_savings_rate'], color='subset')

#### Questions:

Which system parameter allows setting the FEI savings rate?


In [ ]:
# FEI Savings Deposit
fei_savings_rate_process: List[Callable[[Run, Timestep], APR]] = default(
    [lambda _run, _timestep: 0.015]
)

1. What effect does removing the process: **lambda _run, timestep: 0** from the FEI savings rate sweep imply for the model? Would this lead to an interruption in notebook execution?


**A:** this implies removing the scenario in which the FEI savings rate is disabled, hence mimicking the current state of implementation of Fei. In the **FEI Savings Rate analysis** notebook, we are interested in scenarios where the FEI Savings rate is enabled.

This **would lead** to an interruption in the **FEI Savings Rate analysis** notebook execution as there are notebook sections dedicated to **each setting of the savings rate**. It is sufficient to remove the appropriate subsection of the analysis.

2. What effect does adding the process: **lambda _run, timestep: 0.02** to the FEI savings rate sweep imply for the model? Would this lead to an interruption in notebook execution?

**A:** this implies adding an extra scenario for the FEI Savings rate, set at 2%.

This **would NOT lead** to an interruption in the **FEI Savings Rate analysis** notebook execution. However, as there are notebook sections dedicated to **each setting of the savings rate**, one must add the appropriate subsection of the analysis by duplicating cells from other settings of the sweep.

3. _Intuition:_ Inspect the “weight evolution” plots for all settings of the FEI savings rate. Why is the weight for the Idle FEI deposit zero throughout the simulation?

**A:** The weight for Idle FEI (**fei_idle_user_deposit**) is zero, as it is not a yield-bearing deposit. This encodes the dynamic that users have no utility for idle FEI upon the existance of a savings rate competing with other yield bearing avenues - here a money market and a liquidity pool.

4. What are the implications of this on the evolution of capital allocation throughout the simulation?

**A:** the amount of Idle FEI monotonically (with some stochastic noise) reaches zero in all simulation settings and for all runs.

_BONUS:_ how are CAM weights computed?

**A:** CAM weights are represented as a four dimensional vector where each element is one of the four CAM deposits. The weight associated to each deposit is computed as a function of deposit yield and risk, including idle which gets an attribution of zero by current construction.

_BONUS:_ how are CAM weights disaggregated into actual transfers between user deposits? **Hint: [See HackMD](https://hackmd.io/keg-pULfSLqJdqB3dBcahQ)**

CAM weights are disaggregated into constituent transfers which are performed via the **transfer()** function of the User FEI deposit dataclass. Each timestep, the computed weights represent the total change in balance to each deposit. This total change is disaggregated into constituent amounts computed as the solution to a constrained system of linear equations, ensuring total overall balance changes are consistent with indivdual transfers.

**NOTE:** This balance change consistency will not be respected if the deposit does not have enough balance to transfer. In this case all available balance will be transferred.

**Extensions:** For each of the below use only 10 monte carlo runs.

1. Re-run the notebook without the **Idle FEI** CAM deposit. Compare the Capital Allocation Area Chart for savings rate Setting 1 (Section 1.1) with the deposit enabled vs disabled What do you notice? 

    In Section 2.1, compare the **Total User Circulating FEI** plots when the idle deposit is enabled vs disabled.

**A:**

- In Setting 1 (FSR = 2%), without the **Idle FEI User Deposit** the initial conditions of the model make it so that the transitory state within the first couple months is different than when the deposit is present. However, *in percentage terms*, the steady-state of the model is virtually **identical**. This is in accordance with intuiton. When only yield bearing avenues for FEI exist, in the long term, no idle avenues for FEI will be relevant.


- Without the **Idle FEI deposit** changes in user-circulating FEI are **smaller** than with the deposit enabled. The reason for this is that the Idle FEI depoisit starts out with a large amount of FEI which is fully re-allocated amongst other yield bearing deposits very quickly. In absence of the Idle Deposit, capital allocation occurs only between yield bearing deposits, which are initialized at lower values.

2. Re-run the notebook without the **FEI Money Market** CAM deposit. Compare the Capital Allocation Area Chart for savings rate Setting 1 and Setting 4 (disabled). What do you notice? 

**A:**

- In Setting 1 (FSR = 2%), without the fei savings deposit all allocation happens between the **FEI savings deposit** and the **fei liquidity pool**. This model subset is the one that would be representative of a scenario where *Fei completely deleverages from money markets*.


- In Setting 4 (FSR = 0%), we come into an example of a **degenerate analysis**. Here, the capital allocation should be between the **FEI savings deposit** and the **fei liquidity pool**, but the FEI savings rate is disabled, so the CAM in fact reduces **only to capital allocation in liquidity pool**. 

3. Re-run the notebook without the **FEI Savings Deposit** CAM deposit. Compare the Capital Allocation Area Chart for savings rate Setting 1 and Setting 4 (disabled). What do you notice? 

**A:**

- In Setting 1 (FSR = 2%), without the fei savings deposit all allocation happens between the **money market** and the **fei liquidity pool**. In practice, since the **Idle FEI deposit** has do independent dynamics, this model subset is the one *closest to the current state of implementation in the FEI protocol*.

- In Setting 4 (FSR = 0%), there is **no change** compared to Setting 1 or any other setting. In fact, we've **disabled the deposit object of the parameter sweep**, making any rate setting redundant.

4. Re-run the notebook without the **FEI Liquidity Pool** CAM deposit. Compare the Capital Allocation Area Chart for savings rate Setting 1 and Setting 4 (disabled). What do you notice? 

    Then compare **liquidity pool trading fees** (section 3.2). What do you notice?

**A:**

- In Setting 1 (FSR = 2%), without the liquidity pool deposit all allocation happens between the **money market** and the **fei savings deposit**. In practice, since the **Idle FEI deposit** has do independent dynamics, this model subset is the one *closest to the original 'FEI Savings Rate' model*.

- In Setting 4 (FSR = 0%), all allocation is taken by the **money market** deposit, since the **savings deposit** has zero yield, hence zero weight.

- Liquidity Pool trading fees are not CAM-dependent when the deposit is disabled.